In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed=42


import os
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
if torch.cuda.is_available():
    print("Yeah we have a GPU!")
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import numpy as np
from scipy.spatial import distance_matrix

from gdeep.decision_boundary.decision_boundary_calculator import *
from gdeep.create_nets.sample_nn import SampleCNN_MNIST_SAMPLE,\
    SampleCNN_MNIST_SAMPLE_2
from gdeep.create_nets import Net
from gdeep.plotting.visualize_hd_db import LowDimensionalPlane

try:
    from color_scale import lab_color_scale
except:
    pass

ModuleNotFoundError: No module named 'numpy'

# Generating generic CNN

In [ ]:
digit_detect_nn = SampleCNN_MNIST_SAMPLE(smooth=True).to(dev)

print("Trainable parameters of NN:",\
    sum(p.numel() for p in digit_detect_nn.parameters()), '\n')

print('Neural network architecture')
print(digit_detect_nn)

In [ ]:
for param in digit_detect_nn.parameters():
    param.requires_grad = False
model = lambda x: F.softmax(digit_detect_nn(x), dim=-1)[:, 1] #- torch.tensor([0.5])

# Sampling points on DB

In [ ]:
steps = 1000

trajectory = []

dc_samples_random = {'spacial': [], 'gradients': []}

initial_points_batch = torch.rand((600, 1, 28, 28))
labels = model(initial_points_batch) > 0.5
#print(model(initial_points_batch[:1])[0,0])
initial_points_batch.to(dev, non_blocking=True)
g = GradientFlowDecisionBoundaryCalculator(
            model=model,
            initial_points=initial_points_batch,
            optimizer=lambda params: torch.optim.Adam(params)
)
#TODO: clip to [0,1]
for step in range(steps):
    g.step()
    trajectory.append(g.get_decision_boundary().clone().detach()[0])

y = g.get_decision_boundary()
delta = torch.zeros_like(y, requires_grad=True)
loss = torch.sum(model(y+delta))
loss.backward()
dc_samples_random['gradients'].append(delta.grad.detach())

dc_samples_random['spacial'].append(y.detach().cpu())

In [ ]:
dc_samples_random_tensor = {'spacial': [], 'gradients': []}
dc_samples_random_tensor['spacial'] = torch.cat(dc_samples_random['spacial'], axis=0)
dc_samples_random_tensor['gradients'] = torch.cat(dc_samples_random['gradients'], axis=0).reshape([-1, 28*28])
dc_samples_random_tensor['normal'] = dc_samples_random_tensor['gradients'] / dc_samples_random_tensor['gradients'].norm(dim=1)[:, None]

In [ ]:
plt.imshow(dc_samples_random_tensor['gradients'].mean(0).reshape((28, 28)))